In [1]:
# !pip install wonderwords
# !pip install translate

In [107]:
# states = ['start', '1_miss', '2_miss', '3_miss', '4_miss', '5_miss', 'end']
# pics
gallow_symb_list = []
gallow_symb_list.append('0')
gallow_symb_list.append(['____',
                        '|/',
                        '|',
                        '|',
                        '|',
                        '|',
                        'П'])

gallow_symb_list.append(['____',
                        '|/   |',
                        '|',
                        '|',
                        '|',
                        '|',
                        'П'])

gallow_symb_list.append(['____',
                        '|/   |',
                        '|    о',
                        '|',
                        '|',
                        '|',
                        'П'])

gallow_symb_list.append(['____',
                        '|/   |',
                        '|    о',
                        '|    T',
                        '|',
                        '|',
                        'П'])

gallow_symb_list.append(['____',
                        '|/   |',
                        '|    о',
                        '|    T',
                        '|    П',
                        '|',
                        'П'])

gallow_symb_list.append(['____',
                        '|/   |',
                        '|    X',
                        '|    |',
                        '|    П',
                        '|',
                        'П'])

In [86]:
from abc import ABCMeta, abstractmethod, abstractproperty
class Entity():
    __metaclass__=ABCMeta

    @abstractmethod
    def change_state():
        """изменить состояни"""
        pass

In [118]:
class Game(Entity):
    def __init__(self, word):
        self.word = word
        self.state = 'new'
        self.mask = '*'*len(self.word)
        print(f'Угадай слово: {self.mask}\n')
        
    def change_state(self,is_won):
        if is_won == True:
            self.state = 'won'
            print(f'Точно - это слово "{self.word}"! Молодец!')
        else:
            self.state = 'losed'
            epitaph = f'Тебе не удалось угадать слово "{self.word}" ты проиграл.'
            print(epitaph,'✝'*len(epitaph),'\n\n\n', sep='\n')
            
    def open_word(self,latter):
        includes = [i for i, x in enumerate(list(self.word)) if x == latter]
        masked_word = list(self.mask)
        for i in includes:
            masked_word[i] = latter
        self.mask = "".join(masked_word)

In [125]:
# a=Game('good-day')

In [106]:
# https://python-statemachine.readthedocs.io/en/latest/readme.html
# https://github.com/pytransitions/transitions
# !pip install transitions
from transitions import Machine

In [ ]:
class Attempt(object):
    def __init__(self,step,checkedLatterArray,missingLatterArray):
        self.step = 0
        self.checkedLatterArray = {}
        self.missingLatterArray = {}
        
    def new_step(self,latter):
        answer = input(f'\nКакую букву проверяем?')
        if answer in list(self.checkedLatterArray.values()):
            print('Уже была эта буква, укажи другую...')
        else:
            self.step += 1
            print(f'Попытка № {self.step}')
            self.checkedLatterArray[self.step] = latter
            print(f'Попытка № {self.step}')
                
    def check_latter(self,game,latter):
        if latter in game.word:
            print(f'Верно, буква {latter} есть!')
            game.open_word(latter)
            print(f'Продолжаем игру, угадай слово: {game.mask} ?')
        else:
            print(f'Буквы "{latter}" нет в слове {game.mask}\n',
                  AttemptState.self.step() # вряд ли это работет https://habr.com/ru/articles/521946/
                  AttemptState.state
                  f'Попробуй еще, у тебя осталось {6-self.step} попыток...')

In [ ]:
AttemptState = Attempt()
transitions = [
    { 'trigger': 1, 'source': gallow_symb_list[0], 'dest': gallow_symb_list[1]},
    { 'trigger': 2, 'source': gallow_symb_list[1], 'dest': gallow_symb_list[2]},
    { 'trigger': 3, 'source': gallow_symb_list[2], 'dest': gallow_symb_list[3]},
    { 'trigger': 4, 'source': gallow_symb_list[3], 'dest': gallow_symb_list[4]},
    { 'trigger': 5, 'source': gallow_symb_list[4], 'dest': gallow_symb_list[5]},
    { 'trigger': 6, 'source': gallow_symb_list[5], 'dest': gallow_symb_list[6]},
]

# Инициализация машины
machine = Machine(AttemptState, states=gallow_symb_list, transitions=transitions, initial='0')

In [1]:
import requests
import random

In [3]:
# word
from wonderwords import RandomWord
from translate import Translator

def create_word():
    w = RandomWord()
    eng_word = w.word(include_categories=["noun"])
    # sometimes work wrong
    translator= Translator(from_lang="English",to_lang="russian")
    word = translator.translate(eng_word).lower()
    # word
    mask = '*'
    masked_word = mask * len(word)
    return word, mask, masked_word

In [4]:
# defs
def check_the_answer(word, masked_word, answer):
    includes = [i for i, x in enumerate(list(word)) if x == answer]
    masked_word = list(masked_word)
    for i in includes:
        masked_word[i] = answer
    masked_word = "".join(masked_word)    
    return masked_word

def checked(word, answer, masked_word, error_number):
    if error_number != 6:
        print(f'Буквы "{answer}" нет в слове {masked_word}. Попробуй еще, у тебя осталось {6-error_number} попыток...')
        draw_gallow(error_number)
    else:
        draw_gallow(error_number)
        epitaph = f'Тебе не удалось угадать слово "{word}" ты проиграл.'
        print(epitaph,'✝'*len(epitaph),'\n\n\n', sep='\n')
        
def print_stat(error_number, asked_chars):
    print(f'Совершено ошибок: {error_number}\nПроверены буквы: {asked_chars}')

def draw_gallow(error_number):
    for _ in gallow_symb_list[error_number]:
        print(_)

In [5]:
# main def
def attempt(word, masked_word, asked_chars, attempts_counter, error_number):
    answer = input(f'\nКакую букву проверяем?')

    if answer in asked_chars:
        print('Уже была эта буква, укажи другую...') # debug

    else:
        if attempts_counter != 0:
            print(f'Попытка № {attempts_counter+1}')
            asked_chars += ','

        attempts_counter +=1
        asked_chars += answer

        if answer in word:
            if masked_word.count('*') == 0:
                print('Слово разгадано!')
            else:
                masked_word = check_the_answer(word, masked_word, answer)
                print(f'Верно, такая буква есть!')
                print_stat(error_number, asked_chars)
                print(f'Продолжаем игру, угадай слово: {masked_word}')
        else:
            error_number += 1
            print_stat(error_number, asked_chars)
            checked(word, answer, masked_word, error_number)
                    
    return masked_word, answer, asked_chars, attempts_counter, error_number

In [ ]:
# game
if __name__ == '__main__':
    while True:
        for_start = input('ИГРА ВИСЕЛИЦА\nХотите начать игру? \nВведите букву "н":')
        if for_start == 'н':
            word, mask, masked_word = create_word()
            attempts_counter = 0
            error_number = 0
            asked_chars = ''
            print(f'Угадай слово: {mask * len(word)}')
            while error_number < 6:
                masked_word, answer, asked_chars, attempts_counter, error_number = attempt(word, masked_word, asked_chars, attempts_counter, error_number)
        else:
            print('Пока!')
            break